In [6]:
import os
%pwd

'/home/febin/MLOps-AWS-cancer-prediction-system'

## DataIngestion Entity for storing the configurations.

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.cancer_prediction.constants import *
from src.cancer_prediction.utils import read_yaml, create_directories

class ConfigurationManager:

    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH):
            
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import gdown
import zipfile
from src.cancer_prediction import logger
from src.cancer_prediction.utils import get_size

In [13]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self) -> None:

        try:

            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)

            file_id = dataset_url.split("/")[-2]
            logger.info(f"Downloading file from: {dataset_url} into {zip_download_dir}")
            prefix = "https://drive.google.com/uc?/export=download&id="

            gdown.download(f"{prefix}{file_id}", zip_download_dir, quiet=False)
            logger.info(f"Downloaded file: {zip_download_dir}")

        except Exception as e:
            raise e
        

    def extract_zipfile(self) -> None:

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted file into: {unzip_path}")

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()

except Exception as e:
    raise e

2025-04-07 13:19:41,776 - INFO __init__ - yaml file: config/config.yaml loaded successfully
2025-04-07 13:19:41,782 - INFO __init__ - yaml file: params.yaml loaded successfully
2025-04-07 13:19:41,787 - INFO __init__ - created directory at: artifacts
2025-04-07 13:19:41,789 - INFO __init__ - created directory at: artifacts/data_ingestion
2025-04-07 13:19:41,794 - INFO 1614880556 - Downloading file from: https://drive.google.com/file/d/12Y2iiwvbCacjv_CjkFiuuZsOOkM2-2fI/view?usp=sharing into artifacts/data_ingestion/data.zip


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=12Y2iiwvbCacjv_CjkFiuuZsOOkM2-2fI
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=12Y2iiwvbCacjv_CjkFiuuZsOOkM2-2fI&confirm=t&uuid=3bc87b72-d294-4447-b47e-65fd918eb481
To: /home/febin/MLOps-AWS-cancer-prediction-system/artifacts/data_ingestion/data.zip
100%|██████████| 124M/124M [00:16<00:00, 7.67MB/s] 


2025-04-07 13:20:02,884 - INFO 1614880556 - Downloaded file: artifacts/data_ingestion/data.zip
2025-04-07 13:20:05,030 - INFO 1614880556 - Extracted file into: artifacts/data_ingestion
